In [78]:
%pip install evaluate

In [79]:
import evaluate
import pandas as pd
from tqdm import tqdm

bleu = evaluate.load("bleu")
meteor = evaluate.load("meteor")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# Example

In [ ]:
# predictions is a list of sentence
# references is a list of list of sentence

predictions = [ "Birds fly over hills", "hi"]
references = [
   ["Birds fly over mountains"],
   ["hey"]
]

# BLEU

In [ ]:
bleu_score = bleu.compute(predictions=predictions, references=references)
print(bleu_score)

{'bleu': 0.0, 'precisions': [0.6, 0.6666666666666666, 0.5, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 1.0, 'translation_length': 5, 'reference_length': 5}


# METEOR

In [ ]:
meteor_score = meteor.compute(predictions=predictions, references=references)
print(meteor_score)

{'meteor': 0.3680555555555556}


In [80]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [93]:
all_predictions_12k_df = pd.read_csv("/content/drive/MyDrive/ML Caption Craft/ALL_PREDICTIONS_1200.csv")
all_predictions_12k_df.head()


,Unnamed: 0,img_p,caption,preprocessed_captions,base_vit_gpt_caption,base_coca_caption,generated_caption
0,0,1860421016364639460.jpg,Am I the only one who’s still dressing up like...,am i the only one who is still dressing up lik...,a woman wearing a hat standing next to a tree,a woman standing next to a tree in a park .,i am so excited to finally be able to wear thi...
1,1,1862029415539853473.jpg,"End of summer dress, it still feels like summe...",end of summer dress it still feels like summe...,a woman standing in front of a brick wall wear...,a woman standing on the side of a road wearing...,i am so excited to finally be able to share my...
2,2,1867147614090586401.jpg,It’s actually cold enough to wear my camo jack...,it is actually cold enough to wear my camo jac...,a woman standing next to a fence with a hat on,a woman standing on the sidewalk wearing a hat .,i am so excited to finally be able to share my...
3,3,1872175167322844074.jpg,Today is a more appropriate time to pull out m...,today is a more appropriate time to pull out m...,a woman standing in front of a wooden door,a woman leaning against a wall wearing a hat .,i am in love with this dress from it is so s...
4,4,1877240015156943325.jpg,I miss these buttoned up dresses when they wer...,i miss these buttoned up dresses when they wer...,a woman standing in front of a building holdin...,a woman standing in front of a wall holding a ...,i am in love with this dress from it is so ...


In [94]:
all_12k_generated_caption = [] #generated_caption (captionwizard)
all_12k_baseline_ViT_caption = [] #base_vit_gpt_caption
all_12k_baseline_CoCa_caption = [] #base_coca_caption
all_12k_ground_truth = [] #preprocessed_captions

for i , row in tqdm(all_predictions_12k_df.iterrows()):
  all_12k_generated_caption.append(row['generated_caption'])
  all_12k_baseline_ViT_caption.append(row['base_vit_gpt_caption'])
  all_12k_baseline_CoCa_caption.append(row['base_coca_caption'])
  all_12k_ground_truth.append([row['preprocessed_captions']])


1237it [00:00, 16592.64it/s]


In [95]:
'''
bleu_score = bleu.compute(predictions=predictions, references=references)
print(bleu_score)
meteor_score = meteor.compute(predictions=predictions, references=references)
print(meteor_score)
'''
ViT_bleu = bleu.compute(predictions = all_12k_baseline_ViT_caption, references = all_12k_ground_truth)
CoCa_bleu = bleu.compute(predictions = all_12k_baseline_CoCa_caption, references = all_12k_ground_truth)
CaptionWizard_bleu = bleu.compute(predictions = all_12k_generated_caption, references = all_12k_ground_truth)

print("baseline (ViT) model performance (BLEU):", ViT_bleu)
print("baseline (CoCa) model performance (BLEU):", CoCa_bleu)
print("CaptionWizard model performance (BLEU):", CaptionWizard_bleu)

baseline (ViT) model performance (BLEU): {'bleu': 0.00011418164836515014, 'precisions': [0.16279418378054264, 0.006278397356464271, 0.00027603974972396027, 0.00010383137784238397], 'brevity_penalty': 0.04907940929341327, 'length_ratio': 0.24910845982934707, 'translation_length': 13342, 'reference_length': 53559}
baseline (CoCa) model performance (BLEU): {'bleu': 0.0002487053706102207, 'precisions': [0.17097519183491022, 0.0103397341211226, 0.0010959905014156545, 0.0003088962108731466], 'brevity_penalty': 0.05028282890765504, 'length_ratio': 0.25062081069474784, 'translation_length': 13423, 'reference_length': 53559}
CaptionWizard model performance (BLEU): {'bleu': 0.017658099353127135, 'precisions': [0.12198098375383694, 0.02198997982362676, 0.007818697748215048, 0.004635797287827567], 'brevity_penalty': 1.0, 'length_ratio': 2.493885248044213, 'translation_length': 133570, 'reference_length': 53559}


In [96]:
ViT_METEOR = meteor.compute(predictions = all_12k_baseline_ViT_caption, references = all_12k_ground_truth)
CoCa_METEOR = meteor.compute(predictions = all_12k_baseline_CoCa_caption, references = all_12k_ground_truth)
CaptionWizard_METEOR = meteor.compute(predictions = all_12k_generated_caption, references = all_12k_ground_truth)

print("baseline (ViT) model performance (METEOR):", ViT_METEOR)
print("baseline (CoCa) model performance (METEOR):", CoCa_METEOR)
print("CaptionWizard model performance (METEOR):", CaptionWizard_METEOR)

baseline (ViT) model performance (METEOR): {'meteor': 0.03132667664415071}
baseline (CoCa) model performance (METEOR): {'meteor': 0.034501133557871634}
CaptionWizard model performance (METEOR): {'meteor': 0.16221783129504053}


In [97]:
metric_results = {
    "Model" : ["Baseline (ViT-GPT2)", "Baseline (CoCa)", "CaptionWizard"],
    "BLEU score" : [ViT_bleu['bleu'], CoCa_bleu['bleu'], CaptionWizard_bleu['bleu']],
    "METEOR score" : [ViT_METEOR['meteor'], CoCa_METEOR['meteor'], CaptionWizard_METEOR['meteor']],
}

metric_results_df = pd.DataFrame(metric_results)

In [98]:
metric_results_df

,Model,BLEU score,METEOR score
0,Baseline (ViT-GPT2),0.000114,0.031327
1,Baseline (CoCa),0.000249,0.034501
2,CaptionWizard,0.017658,0.162218
